### Scrapeando la página Toc-Toc con Selenium
![Logo Toc-Toc](https://d1cfu8v5n1wsm.cloudfront.net/toctoc/img/logo-toctoc-200x200.jpg)

Utilizaremos Selenium para poder scrapear la página de *www.toctoc.com*, desde la cual, se obtendrán todos los detalles (valor, m2, nº de habitaciones, nº de baños, etc) de las propiedades usadas en venta(casas y departamentos) que se encuentren en la Región Metropolitana hasta el día de hoy.

### Selenium
Selenium es una herramienta que nos permitirá controlar un navegador y poder utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto, vamor a importar el módulo webdriver y el módulo keys (para poder indicar atajos del teclado para manipular el navegador remotamente)

In [102]:
from selenium import webdriver
#Para generar eventos de click
from selenium.webdriver.common.keys import Keys
#Para demoras dinámicas
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
#Para convertir listas a dataframe
import pandas as pd
#Para medir tiempo de ejecución de funciones
from time import time
from datetime import datetime
import sys

**Paso 1**: Instanciar un driver del navegador (en este caso, de Chrome). Asegurarse de que el archivo driver del navegador seleccionado se encuentre en la carpeta que se especifica en el path.

In [1207]:
options = webdriver.ChromeOptions() #Generamos una variable de opción vacía.
options.add_argument('--incognito') #le agregamos a la variable creada un argumento, en este caso de incógnito.
driver = webdriver.Chrome(executable_path= '/Users/danielparedesvillalobos/opt/anaconda3/chromedriver', options=options) #Le agregamos la variable creada al argumento options.

**Paso 2**: Generamos una variable con la url en cuestión y le inicamos al navegador que la carge (mediante webdriver)

In [1208]:
url_main = 'https://www.toctoc.com/resultados/lista/compra/casa-departamento/?moneda=2&precioDesde=0&precioHasta=0&dormitoriosDesde=&dormitoriosHasta=&banosDesde=0&banosHasta=0&estado=2&disponibilidadEntrega=&numeroDeDiasTocToc=0&superficieDesdeUtil=0&superficieHastaUtil=0&superficieDesdeConstruida=0&superficieHastaConstruida=0&superficieDesdeTerraza=0&superficieHastaTerraza=0&superficieDesdeTerreno=0&superficieHastaTerreno=0&ordenarPor=0&pagina=1&paginaInterna=1&zoom=15&idZonaHomogenea=0&atributos=&texto=Regi%C3%B3n%20Metropolitana%20De%20Santiago,%20Chile&viewport=-34.290931418749516,-71.83796149148834,-32.92240654382668,-69.64697082118785&idPoligono=2240&publicador=0&temporalidad=0'
driver.get(url_main)

**Paso 3**: Extraemos la información de interés.

In [1233]:
#Obtenemos los elementos de la lista de resultados (sólo la página actual)
propiedades = driver.find_elements_by_xpath('//li[@class="un-ress tp1"]')

In [1234]:
#Consultamos un sólo resultado de la lista para evaluar su composición
propiedad = propiedades[0]

In [1235]:
#Obtenemos la url de interés para el resultado que evaluamos (sólo el primero) y la abrimos en una nueva pestaña
pagina = propiedad.find_element_by_xpath('./a[@class="lnk-info"]')
pagina.send_keys(Keys.COMMAND + Keys.RETURN)



In [1236]:
# Hacemos foco sobre la nueva pestaña
driver.switch_to.window(driver.window_handles[1])

In [1246]:
# Cerrar la nueva pestaña de URL-secundaria
driver.close()

**Paso 3.1**: Generamos las funciones para realizar la extracción de datos

In [73]:
def obtener_info_propiedad(driver):
    '''
    Función que retorna un diccionario con las variables de interés de cada una de las propiedades de la página.
    Nota: El valor de la propiedad no se representará en la misma moneda por cada columna, 
    esto es algo que se debe limpiar después en la próxima etapa.
    '''
    #Obtenemos el código de la propiedad
    try:
        cod_propiedad = driver.find_element_by_xpath('//li[@class="cod"]').text
    except:
        cod_propiedad = ''
        
    #Obtenemos el tipo de propiedad
    try:
        if 'Departamento' in driver.find_element_by_xpath('//div[@class="info-cabecera-ficha"]').text:
            tipo_propiedad = 'Departamento'
        elif 'Casa' in driver.find_element_by_xpath('//div[@class="info-cabecera-ficha"]').text: 
            tipo_propiedad = 'Casa'
        else:
            tipo_propiedad = ''
    except:
        tipo_propiedad = ''
        
    #Obtenemos información general
    try:    
        info_general = driver.find_element_by_xpath('//div[@class="info-cabecera-ficha"]').text
    except:
        info_general = ''
        
    #Obtenemos el nombre principal
    try:    
        nombre_propiedad = driver.find_element_by_xpath('//h1[@class="tt-ficha"]').text
    except:
        nombre_propiedad =''
        
    #Obtenemos el precio A de la propiedad
    try:
        precio_a = driver.find_element_by_xpath('//div[@class="precio-b"]/strong').text
    except:
        precio_a=''
        
    #Obtenemos el precio B de la propiedad
    try:    
        precio_b = driver.find_element_by_xpath('//em[@class="precioAlternativo"]/strong').text
    except:
        precio_b =''
        
    #Obtenemos número de dormitorios
    try:
        n_dormitorios = driver.find_element_by_xpath('//*[@id="informacionBasica"]/div[2]/ul/li[2]/strong').text
    except:
        n_dormitorios =''
        
    #Obtenemos número de baños
    try:
        n_banos = driver.find_element_by_xpath('//li[@class="baños"]/strong').text[0]
    except:
        n_banos =''

    #Obtenemos superficie de terreno
    try:
        sup_terreno = driver.find_element_by_xpath('//li[@class="metrosTerreno"]/strong').text
    except:
        sup_terreno =''

    #Obtenemos superficie util
    try:
        sup_util = driver.find_element_by_xpath('//li[@class="metrosUtiles"]/strong').text
    except:
        sup_util = ''

    #Obtenemos superficie construida
    try:
        sup_construida = driver.find_element_by_xpath('//li[@class="metrosConstruidos"]/strong').text
    except:
        sup_construida =''

    #Obtenemos avaluo fiscal
    try:
        aval_fiscal = driver.find_element_by_xpath('//strong[@id="lblAvaluo"]').text
    except:
        aval_fiscal =''

    #Obtenemos fecha de publicación
    try:
        fecha_publicacion = driver.find_element_by_xpath('//*[@id="informacionBasica"]/div[2]/ul/li[16]/strong').text
    except:
        fecha_publicacion =''

    #Obtenemos última plusvalia del sector (fecha)
    try:
        fecha_plusvalia = driver.find_elements_by_xpath('//*[@id="chartPlusvalia"]/div/div[1]/div/div/table/tbody/tr/td[1]')[-1].get_attribute('innerHTML')
    except:
        fecha_plusvalia =''
        
    #Obtenemos última plusvalia del sector (valor)
    try:
        valor_plusvalia = driver.find_elements_by_xpath('//*[@id="chartPlusvalia"]/div/div[1]/div/div/table/tbody/tr/td[2]')[-1].get_attribute('innerHTML')
    except:
        valor_plusvalia =''
        
    #Obtenemos ubicación (latitud)
    try:
        latitud = driver.find_element_by_xpath('//input[@id="h_br_lat"]').get_attribute('value')
    except:
        latitud =''
        
    #Obtenemos ubicación (longitud)
    try:
        longitud = driver.find_element_by_xpath('//input[@id="h_br_lon"]').get_attribute('value')
    except:
        longitud =''
        
    info_propiedad = {'cod_propiedad':cod_propiedad,'tipo_propiedad':tipo_propiedad,'info_general':info_general,'nombre_propiedad':nombre_propiedad,
                      'precio_a':precio_a,'precio_b':precio_b,'n_dormitorios':n_dormitorios,'n_banos':n_banos,'sup_terreno':sup_terreno,
                      'sup_util':sup_util,'sup_construida':sup_construida,'aval_fiscal':aval_fiscal,'fecha_publicacion':fecha_publicacion,'fecha_plusvalia':fecha_plusvalia,'valor_plusvalia':valor_plusvalia,
                      'latitud':latitud,'longitud':longitud}
    print(f'   Se escrapea propiedad {nombre_propiedad}')
    return info_propiedad

In [93]:
def obtener_informacion(driver):
    '''
    Función que retorna una lista con todos los diccionarios de las variables obtenidas de cada una de las propiedades.
    '''
    hora_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    start_time = time() #Comienza conteo de tiempo
    print(f'El scraping se comienza a ejecutar a las {hora_start}')
    pag= int(driver.find_element_by_xpath('//li[@class="page-item active"]').text)
    prop = 26035 #Nº de propiedad con que comienza la descarga (sólo referencial, no condiciona la programación)
    info = []
    while driver.find_element_by_xpath('//a[@aria-label="Next"]'):
        try:
            propiedades = driver.find_elements_by_xpath('//li[@class="un-ress tp1"]')
            print(f'Se encontraron {len(propiedades)} propiedades en la página nº{pag}')#
            print(f'Inicializando scraping de la página nº{pag}...') #
            for propiedad in propiedades:
                propiedad.find_element_by_xpath('./a[@class="lnk-info"]').send_keys(Keys.COMMAND + Keys.RETURN)
                driver.switch_to.window(driver.window_handles[1])
                delay = 10                                                     
                try:
                    #Introducimos una demora dinámica
                    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="partialCabecera"]/div[1]/header/h1')))#//*[@id="hFicha"]#'/html/body/footer/div/div[1]/div[1]/h4'
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print(f'    La página de la propiedad nº{prop} terminó de cargar')#
                    print(f'   Nº{prop}...inicializando scraping')                                    
                    try:
                        detalle_propiedad = obtener_info_propiedad(driver)
                    except:
                        print(f'    No fue posible scrapear la propiedad nº{prop}') 
                    prop +=1       
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except TimeoutException:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    print(f'Nº{prop}...la página de la propiedad tardó demasiado en cargar')
                    #print(f'    No fue posible scrapear la propiedad nº{prop}')#
                    #detalle_propiedad = []
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print('     La página presenta algún problema')#
                    print(f'Nº{prop}...no fue posible scrapear la propiedad ') 
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                info.append(detalle_propiedad)
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
            pag +=1
            if pag ==1167:
                break
            try:
                #Introducimos una demora dinámica para la nueva página
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//li[@class="page-item active"]')))
                print('La siguiente página terminó de cargar')
            except TimeoutException:
                print('La página tardó demasiado en cargar')
                return info
        except:
            print(f'El escraping se ha detenido inesperadamente')#,sys.exc_info()[0]
            
            hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
            print(f'Se alcanzaron a escrapear {prop-1-16756} propiedades')
            print(f'Se retornarán los valores obtenidos hasta el momento')
            print(f'El scraping se comienza a ejecutar a las {hora_start}')
            print(f'El scraping se termina de ejecutar a las {hora_end}')
            elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
            print("Tiempo de ejecución: %.3f seconds." % elapsed_time)
            return info
        #finally:
             
    else:
        print("El scraping ha finalizado")
    
    print(f'Se escrapearon {prop-1-16756} propiedades')
    hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    print(f'El scraping se comienzó a ejecutar a las {hora_start}')
    print(f'El scraping se termina de ejecutar a las {hora_end}')
    elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
    print("Tiempo de ejecución: %.3f seconds." % elapsed_time)   
    return info
    

In [47]:
def obtener_informacion_arriendo(driver):
    '''
    Función que retorna una lista con todos los diccionarios de las variables obtenidas de cada una de las propiedades.
    '''
    hora_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    start_time = time() #Comienza conteo de tiempo
    print(f'El scraping se comienza a ejecutar a las {hora_start}')
    pag= int(driver.find_element_by_xpath('//li[@class="page-item active"]').text)
    prop = 1 #Nº de propiedad con que comienza la descarga (sólo referencial, no condiciona la programación)
    info = []
    while driver.find_element_by_xpath('//a[@aria-label="Next"]'):
        try:
            propiedades = driver.find_elements_by_xpath('//li[@class="un-ress tp3"]')
            print(f'Se encontraron {len(propiedades)} propiedades en la página nº{pag}')#
            print(f'Inicializando scraping de la página nº{pag}...') #
            for propiedad in propiedades:
                propiedad.find_element_by_xpath('./a[@class="lnk-info"]').send_keys(Keys.COMMAND + Keys.RETURN)
                driver.switch_to.window(driver.window_handles[1])
                delay = 10                                                     
                try:
                    #Introducimos una demora dinámica
                    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="partialCabecera"]/div[1]/header/h1')))#//*[@id="hFicha"]#'/html/body/footer/div/div[1]/div[1]/h4'
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print(f'    La página de la propiedad nº{prop} terminó de cargar')#
                    print(f'   Nº{prop}...inicializando scraping')                                    
                    try:
                        detalle_propiedad = obtener_info_propiedad(driver)
                    except:
                        print(f'    No fue posible scrapear la propiedad nº{prop}') 
                    prop +=1       
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except TimeoutException:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    print(f'Nº{prop}...la página de la propiedad tardó demasiado en cargar')
                    #print(f'    No fue posible scrapear la propiedad nº{prop}')#
                    #detalle_propiedad = []
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    #print(f'Inicializando carga de página propiedad nº{prop}')#
                    #print('     La página presenta algún problema')#
                    print(f'Nº{prop}...no fue posible scrapear la propiedad ') 
                    prop +=1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                info.append(detalle_propiedad)
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
            pag +=1
            if pag ==235:
                break
            try:
                #Introducimos una demora dinámica para la nueva página
                WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//li[@class="page-item active"]')))
                print('La siguiente página terminó de cargar')
            except TimeoutException:
                print('La página tardó demasiado en cargar')
                return info
            except:
                print('El scraping ha finalizado...')
        except:
            print(f'El escraping se ha detenido inesperadamente')#,sys.exc_info()[0]
            
            hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
            print(f'Se alcanzaron a escrapear {prop-1} propiedades')
            print(f'Se retornarán los valores obtenidos hasta el momento')
            print(f'El scraping se comienza a ejecutar a las {hora_start}')
            print(f'El scraping se termina de ejecutar a las {hora_end}')
            elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
            print("Tiempo de ejecución: %.3f seconds." % elapsed_time)
            return info
        #finally:
             
    else:
        print("El scraping ha finalizado")
        
    
    print(f'Se escrapearon {prop-1} propiedades')
    hora_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #Hora comienza función
    print(f'El scraping se comienzó a ejecutar a las {hora_start}')
    print(f'El scraping se termina de ejecutar a las {hora_end}')
    elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
    print("Tiempo de ejecución: %.3f seconds." % elapsed_time)   
    return info
    

In [92]:
options = webdriver.ChromeOptions() #Generamos una variable de opción vacía.
options.add_argument('--incognito') #le agregamos a la variable creada un argumento, en este caso de incógnito.
options.add_argument('--headless') #Para ejecutar en segundo plano
options.add_argument('--no-sandbox')#Para evitar crasheo de la página
options.add_argument('--disable-dev-shm-usage')#Para grandes cantidades de datos que requieren grandes cantidades de memoria (Mejor deshabilitar)
#options.add_argument('--window-size=WINDOW_SIZE') #Para manejar el tamaño de la ventana de chrome
driver = webdriver.Chrome(executable_path= '/Users/danielparedesvillalobos/opt/anaconda3/chromedriver', options=options) #Le agregamos la variable creada al argumento options.

In [94]:
#Para venta de propiedades
pag_inicial = 904
url_main = f'https://www.toctoc.com/resultados/lista/compra/casa-departamento/metropolitana/?moneda=2&precioDesde=0&precioHasta=0&dormitoriosDesde=&dormitoriosHasta=&banosDesde=0&banosHasta=0&estado=2&disponibilidadEntrega=&numeroDeDiasTocToc=0&superficieDesdeUtil=0&superficieHastaUtil=0&superficieDesdeConstruida=0&superficieHastaConstruida=0&superficieDesdeTerraza=0&superficieHastaTerraza=0&superficieDesdeTerreno=0&superficieHastaTerreno=0&ordenarPor=0&pagina=1&paginaInterna={pag_inicial}&zoom=15&idZonaHomogenea=0&atributos=&texto=Regi%C3%B3n%20Metropolitana%20De%20Santiago,%20Chile&viewport=-34.290931418749516,-71.83796149148834,-32.92240654382668,-69.64697082118785&idPoligono=2240&publicador=0&temporalidad=0#'
driver.get(url_main)

In [66]:
#Para arriendos de propiedades
pag_inicial = 31
url_arriendos = f'https://www.toctoc.com/resultados/lista/arriendo/casa-departamento/metropolitana/?moneda=2&precioDesde=0&precioHasta=0&dormitoriosDesde=&dormitoriosHasta=&banosDesde=0&banosHasta=0&estado=0&disponibilidadEntrega=&numeroDeDiasTocToc=0&superficieDesdeUtil=0&superficieHastaUtil=0&superficieDesdeConstruida=0&superficieHastaConstruida=0&superficieDesdeTerraza=0&superficieHastaTerraza=0&superficieDesdeTerreno=0&superficieHastaTerreno=0&ordenarPor=0&pagina=1&paginaInterna={pag_inicial}&zoom=15&idZonaHomogenea=0&atributos=&texto=Región%20Metropolitana%20De%20Santiago,%20Chile&viewport=-34.29093141875031,-71.83796149148732,-32.9224065438275,-69.64697082118687&idPoligono=2240&publicador=0&temporalidad=0#'
driver.get(url_arriendos)

In [95]:
#start_time = time() #Comienza conteo de tiempo
seg_904_fin = obtener_informacion(driver)#desde página 594 y 16756 propiedades...tiempo indeterminado
#elapsed_time = time() - start_time #Genera variable con la diferencia entre el comienzo y el término de la función
#driver.close()
#print("Elapsed time: %.3f seconds." % elapsed_time)

El scraping se comienza a ejecutar a las 2021-07-03 21:01:19
Se encontraron 30 propiedades en la página nº904
Inicializando scraping de la página nº904...
   Nº26035...inicializando scraping
   Se escrapea propiedad Las Brisas Norte
   Nº26036...inicializando scraping
   Se escrapea propiedad Camino Gran Vista
   Nº26037...inicializando scraping
   Se escrapea propiedad El Remanso / San Carlos de Apoquindo.
   Nº26038...inicializando scraping
   Se escrapea propiedad La Foresta
   Nº26039...inicializando scraping
   Se escrapea propiedad Camino Los Refugios - El Arrayán
   Nº26040...inicializando scraping
   Se escrapea propiedad Condominio Los Quillayes
   Nº26041...inicializando scraping
   Se escrapea propiedad CARLOS SILVA VILDOSOLA
   Nº26042...inicializando scraping
   Se escrapea propiedad Nicanor Plaza - Helsby
   Nº26043...inicializando scraping
   Se escrapea propiedad Padre Hurtado/Valenzuela Puelma
   Nº26044...inicializando scraping
   Se escrapea propiedad Helsby
   Nº260

   Nº26126...inicializando scraping
   Se escrapea propiedad Chicureo Poniente
   Nº26127...inicializando scraping
   Se escrapea propiedad Mall Vivo Los Trapenses
   Nº26128...inicializando scraping
   Se escrapea propiedad Cerro Pan de Azucar/Manquehue Oriente
   Nº26129...inicializando scraping
   Se escrapea propiedad Manquehe Oriente
   Nº26130...inicializando scraping
   Se escrapea propiedad Lumen/Segunda Transversal
   Nº26131...inicializando scraping
   Se escrapea propiedad Walker Martinez
   Nº26132...inicializando scraping
   Se escrapea propiedad WALKER MARTINEZ (AG)/GERONIMO ALDERETE
   Nº26133...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna / Lluta
   Nº26134...inicializando scraping
   Se escrapea propiedad Av. Dorsal/Av. El Salto
   Nº26135...inicializando scraping
   Se escrapea propiedad AV. LAS NIEVES ORIENTE / EL PERAL
   Nº26136...inicializando scraping
   Se escrapea propiedad Colegio San José de Chicureo
   Nº26137...inicializando scraping
   S

   Nº26217...inicializando scraping
   Se escrapea propiedad Plaza Ossadon-Echeñique
   Nº26218...inicializando scraping
   Se escrapea propiedad ECHEÑIQUE
   Nº26219...inicializando scraping
   Se escrapea propiedad Carlos Ossandon
   Nº26220...inicializando scraping
   Se escrapea propiedad Cercana a Estación Metro Eduardo Castillo Castillo Velasco
   Nº26221...inicializando scraping
   Se escrapea propiedad José Manuel Infante
   Nº26222...inicializando scraping
   Se escrapea propiedad Chicureo Poniente, Condominio El Algarrobal I, sitio
   Nº26223...inicializando scraping
   Se escrapea propiedad 
   Nº26224...inicializando scraping
   Se escrapea propiedad Chicureo Poniente
   Nº26225...inicializando scraping
   Se escrapea propiedad Chicureo Poniente
   Nº26226...inicializando scraping
   Se escrapea propiedad Chicureo Poniente
   Nº26227...inicializando scraping
   Se escrapea propiedad Av. Eyzaguirre y Santa Av. Rosa
   Nº26228...inicializando scraping
   Se escrapea propiedad

   Se escrapea propiedad Paradero 13 Avenida Gabriela poniente
   Nº26311...inicializando scraping
   Se escrapea propiedad Juan Moya / Castillo Velasco
   Nº26312...inicializando scraping
   Se escrapea propiedad lider express
   Nº26313...inicializando scraping
   Se escrapea propiedad Los Queltehues
   Nº26314...inicializando scraping
   Se escrapea propiedad LOMAS DE LO AGUIRRE
   Nº26315...inicializando scraping
   Se escrapea propiedad Tabancura
   Nº26316...inicializando scraping
   Se escrapea propiedad Tabancura
   Nº26317...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº26318...inicializando scraping
   Se escrapea propiedad Piedra Roja
   Nº26319...inicializando scraping
   Se escrapea propiedad Piedra Roja
   Nº26320...inicializando scraping
   Se escrapea propiedad Maravillosa Casa de 3 pisos (421/600) - La Parva Alta
   Nº26321...inicializando scraping
   Se escrapea propiedad LA PARVA ALTA
   Nº26322...inicializando scraping
   Se escrapea propiedad Caramp

   Se escrapea propiedad Kennedy / Preciosa vista
   Nº26403...inicializando scraping
   Se escrapea propiedad Hermanos Cabot / Capitan Crosbie
   Nº26404...inicializando scraping
   Se escrapea propiedad HERMANOS CABOT
   Nº26405...inicializando scraping
   Se escrapea propiedad Geronimo de Alderete con kennedy
   Nº26406...inicializando scraping
   Se escrapea propiedad 
   Nº26407...inicializando scraping
   Se escrapea propiedad Hermanos Cabot - Vista Panoramica
   Nº26408...inicializando scraping
   Se escrapea propiedad Departamento Mariposa - Las Verbenas
   Nº26409...inicializando scraping
   Se escrapea propiedad Hermanos Cabot
   Nº26410...inicializando scraping
   Se escrapea propiedad HERMANOS CABOT
   Nº26411...inicializando scraping
   Se escrapea propiedad Capitán Crosbie - Hermanos Cabot
   Nº26412...inicializando scraping
   Se escrapea propiedad Cerro La Parva
   Nº26413...inicializando scraping
   Se escrapea propiedad Kennedy
   Nº26414...inicializando scraping
   S

...

   Se escrapea propiedad Tomas Moro
   Nº32617...inicializando scraping
   Se escrapea propiedad Oxford /Camino El Alba
   Nº32618...inicializando scraping
   Se escrapea propiedad 
   Nº32619...inicializando scraping
   Se escrapea propiedad Los Maitenes
   Nº32620...inicializando scraping
   Se escrapea propiedad Proyecto macul 3401
   Nº32621...inicializando scraping
   Se escrapea propiedad Pedro De Valdivia / Agrícola
   Nº32622...inicializando scraping
   Se escrapea propiedad VILLARRICA
   Nº32623...inicializando scraping
   Se escrapea propiedad Nonato Coo con Domingo Tocornal
Nº32624...la página de la propiedad tardó demasiado en cargar
   Nº32625...inicializando scraping
   Se escrapea propiedad Liray Lo pinto
   Nº32626...inicializando scraping
   Se escrapea propiedad Golf de Manquehue
   Nº32627...inicializando scraping
   Se escrapea propiedad Los Trapenses
   Nº32628...inicializando scraping
   Se escrapea propiedad Gran Vista
   Nº32629...inicializando scraping
   Se es

   Nº32717...inicializando scraping
   Se escrapea propiedad (Hacienda Chicureo)
   Nº32718...inicializando scraping
   Se escrapea propiedad Piedra Roja
   Nº32719...inicializando scraping
   Se escrapea propiedad Chicureo / Piedra Roja
   Nº32720...inicializando scraping
   Se escrapea propiedad Piedra Roja
   Nº32721...inicializando scraping
   Se escrapea propiedad (AR) Las Araucarias/Parque Puyehue
   Nº32722...inicializando scraping
   Se escrapea propiedad Plaza Castelar/ Isabel la Católica
   Nº32723...inicializando scraping
   Se escrapea propiedad Camino el Bosque con Camino Melipilla
   Nº32724...inicializando scraping
   Se escrapea propiedad GUILLERMO FRANKE
La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº1127
Inicializando scraping de la página nº1127...
   Nº32725...inicializando scraping
   Se escrapea propiedad 
   Nº32726...inicializando scraping
   Se escrapea propiedad Pje San Luis (AR)/Madre Bonifacia Rodríguez
   Nº32727...inicia

   Se escrapea propiedad El Rosal
   Nº32812...inicializando scraping
   Se escrapea propiedad Maipú la reforma
   Nº32813...inicializando scraping
   Se escrapea propiedad 
   Nº32814...inicializando scraping
   Se escrapea propiedad Av. Carlos Valdovinos
La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº1130
Inicializando scraping de la página nº1130...
   Nº32815...inicializando scraping
   Se escrapea propiedad A. Escuela Agrícola
   Nº32816...inicializando scraping
   Se escrapea propiedad Avenida Vicuña Mackenna
   Nº32817...inicializando scraping
   Se escrapea propiedad 
   Nº32818...inicializando scraping
   Se escrapea propiedad Condominio Los Montes / Piedra Roja
   Nº32819...inicializando scraping
   Se escrapea propiedad CHICUREO / AERÓDROMO
   Nº32820...inicializando scraping
   Se escrapea propiedad Condominio Los Montes / Piedra Roja
   Nº32821...inicializando scraping
   Se escrapea propiedad 
   Nº32822...inicializando scraping
   Se e

La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº1133
Inicializando scraping de la página nº1133...
   Nº32905...inicializando scraping
   Se escrapea propiedad Santiago Benitez/Alfredo Salgado
   Nº32906...inicializando scraping
   Se escrapea propiedad san jose
   Nº32907...inicializando scraping
   Se escrapea propiedad Metro ULA/ Barrio República
   Nº32908...inicializando scraping
   Se escrapea propiedad Hacienda Los Conquistadores / Av. San Carlos
   Nº32909...inicializando scraping
   Se escrapea propiedad Monte Sinaí con Draco, Puente Alto
   Nº32910...inicializando scraping
   Se escrapea propiedad Chacabuco/ Hnos. Carrera
   Nº32911...inicializando scraping
   Se escrapea propiedad Camino la Cumbre/ Valle la Dehesa
   Nº32912...inicializando scraping
   Se escrapea propiedad (MR)AV MÉXICO/AV EL SALTO
   Nº32913...inicializando scraping
   Se escrapea propiedad Victor Rae - Manquehue
   Nº32914...inicializando scraping
   Se escrapea propieda

   Nº32996...inicializando scraping
   Se escrapea propiedad Los Clarines, Los Juncos, Jardín de lo Prado
   Nº32997...inicializando scraping
   Se escrapea propiedad Renaico
   Nº32998...inicializando scraping
   Se escrapea propiedad Joaquín Edwards Bello/Pedro Lira
   Nº32999...inicializando scraping
   Se escrapea propiedad Socoroma/ Gabriela Figueroa
   Nº33000...inicializando scraping
   Se escrapea propiedad GRAL SAN MARTIN(AG)/LO SECO
   Nº33001...inicializando scraping
   Se escrapea propiedad Condominio El Algarrobal de Batuco
   Nº33002...inicializando scraping
   Se escrapea propiedad Aurora de Chile/San Jose de la Estrella
   Nº33003...inicializando scraping
   Se escrapea propiedad (VR) CALLE MANUEL RODRIGUEZ/(VR) PJE MANUEL BULNES
   Nº33004...inicializando scraping
   Se escrapea propiedad Comunidad Las Camelias
   Nº33005...inicializando scraping
   Se escrapea propiedad AV. CALERA DE TANGO(AG)/CAMINO CATEMITO
   Nº33006...inicializando scraping
   Se escrapea propieda

In [67]:
#Para arriendo
arr_seg_31_fin = obtener_informacion_arriendo(driver)
driver.close()

El scraping se comienza a ejecutar a las 2021-07-02 00:16:04
Se encontraron 30 propiedades en la página nº31
Inicializando scraping de la página nº31...
   Nº1...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº2...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº3...inicializando scraping
   Se escrapea propiedad Vitacura.
   Nº4...inicializando scraping
   Se escrapea propiedad Armando Jaramillo
   Nº5...inicializando scraping
   Se escrapea propiedad Nueva Costanera
   Nº6...inicializando scraping
   Se escrapea propiedad Alonso de Córdova
   Nº7...inicializando scraping
   Se escrapea propiedad Las Catalpas/Vitacura
   Nº8...inicializando scraping
   Se escrapea propiedad Las Nieves
   Nº9...inicializando scraping
   Se escrapea propiedad ALONSO DE CORDOVA - VITACURA/LAS NIEVES
   Nº10...inicializando scraping
   Se escrapea propiedad Departamento Amoblado Las Nieves
   Nº11...inicializando scraping
   Se escrapea propiedad Las Nieves /Las Catalpas
   Nº12

   Nº95...inicializando scraping
   Se escrapea propiedad Moderno y completo departamento / Metro I. de Suárez
   Nº96...inicializando scraping
   Se escrapea propiedad Barros Borgoño / Manuel Montt
   Nº97...inicializando scraping
   Se escrapea propiedad Eleodoro Yáñez / Antonio Varas
   Nº98...inicializando scraping
   Se escrapea propiedad ELIODORO YAÑEZ / FEDERICO FROEBEL
   Nº99...inicializando scraping
   Se escrapea propiedad HUERFANOS / AMUNÁTEGUI
   Nº100...inicializando scraping
   Se escrapea propiedad Metro La Moneda-Paseo Bulnes
   Nº101...inicializando scraping
   Se escrapea propiedad METRO SANTA ANA
   Nº102...inicializando scraping
   Se escrapea propiedad Av Manuel Rodríguez y Tucapel Jiménez/Moneda - Santiago Centro
   Nº103...inicializando scraping
   Se escrapea propiedad Eleuterio Ramírez con San Diego
   Nº104...inicializando scraping
   Se escrapea propiedad Lord Cochrane con Tarapacá
   Nº105...inicializando scraping
   Se escrapea propiedad Metro La Moneda
  

   Nº190...inicializando scraping
   Se escrapea propiedad Av. Kennedy, Gerónimo de Alderete
   Nº191...inicializando scraping
   Se escrapea propiedad CAMOENS 6470 / VITACURA
   Nº192...inicializando scraping
   Se escrapea propiedad av kennedy
   Nº193...inicializando scraping
   Se escrapea propiedad Kennedy lateral / Oscar Tenhann
   Nº194...inicializando scraping
   Se escrapea propiedad 10 de Julio Av Matta
   Nº195...inicializando scraping
   Se escrapea propiedad Metro Santa Isabel
   Nº196...inicializando scraping
   Se escrapea propiedad Se arrienda departamento cerca de todo
   Nº197...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna
   Nº198...inicializando scraping
   Se escrapea propiedad Portugal / Copiapo
   Nº199...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna/10 de Juliio
   Nº200...inicializando scraping
   Se escrapea propiedad Argomedo
   Nº201...inicializando scraping
   Se escrapea propiedad Fray Camilo Henríquez / Líder Express
 

   Nº285...inicializando scraping
   Se escrapea propiedad Portugal con Marín
   Nº286...inicializando scraping
   Se escrapea propiedad General Jofré con Fray Camilo Henriquez
   Nº287...inicializando scraping
   Se escrapea propiedad Metro Santa Isabel
   Nº288...inicializando scraping
   Se escrapea propiedad General Jofre - Vicuña Mackenna
   Nº289...inicializando scraping
   Se escrapea propiedad Lira
   Nº290...inicializando scraping
   Se escrapea propiedad Metro Parque Bustamante
   Nº291...inicializando scraping
   Se escrapea propiedad Santa Isabel / Lira
   Nº292...inicializando scraping
   Se escrapea propiedad Vicuña Mackenna entre Bilbao y Marín
   Nº293...inicializando scraping
   Se escrapea propiedad EXCELENTE CONECTIVIDAD
   Nº294...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº295...inicializando scraping
   Se escrapea propiedad :titulo:
   Nº296...inicializando scraping
   Se escrapea propiedad Santa Isabel/Vicuña Mackenna
   Nº297...inicializando s

   Nº380...inicializando scraping
   Se escrapea propiedad SAN FRANCISCO DE ASIS / FRAY BERNARDO
   Nº381...inicializando scraping
   Se escrapea propiedad Las Condes - AMOBLADO
   Nº382...inicializando scraping
   Se escrapea propiedad CONDOMINIO. MIRADOR DE SAN DAMIAN
   Nº383...inicializando scraping
   Se escrapea propiedad San Carlos de Apoquindo.
   Nº384...inicializando scraping
   Se escrapea propiedad Casa en arriendo condominio exclusivo.( opción amoblada)
   Nº385...inicializando scraping
   Se escrapea propiedad GRAN VÍA / VÍA ROJA
   Nº386...inicializando scraping
   Se escrapea propiedad San Jorge - Francisco de Villagra
   Nº387...inicializando scraping
   Se escrapea propiedad Irarrázaval
   Nº388...inicializando scraping
   Se escrapea propiedad Plaza Egaña
   Nº389...inicializando scraping
   Se escrapea propiedad Irarrázaval / Heriberto Covarrubias
   Nº390...inicializando scraping
   Se escrapea propiedad Los Aliaga con Duble Almeyda
La siguiente página terminó de c

...

   Nº5634...inicializando scraping
   Se escrapea propiedad Carmen 187
   Nº5635...inicializando scraping
   Se escrapea propiedad Metro Santa Lucía - Barrio Londres
La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº219
Inicializando scraping de la página nº219...
   Nº5636...inicializando scraping
   Se escrapea propiedad Agustinas
   Nº5637...inicializando scraping
   Se escrapea propiedad ¡¡¡San Isidro entre Tarapaca y Marin!!!!CON COCINA CERRA Y LOGIA*
   Nº5638...inicializando scraping
   Se escrapea propiedad CURICO
   Nº5639...inicializando scraping
   Se escrapea propiedad Colegio Los Alerces
   Nº5640...inicializando scraping
   Se escrapea propiedad NUEVO SIN USO EL TRANQUE/ LO BARNECHEA
   Nº5641...inicializando scraping
   Se escrapea propiedad EL TRANQUE/ BEAUTIFUL/ BRAND NEW/ YEAR 2019
   Nº5642...inicializando scraping
   Se escrapea propiedad Clasico 4 dormitorios+ Servicios* Frente al MALL LA DEHESA*
   Nº5643...inicializando scraping
 

   Nº5726...inicializando scraping
   Se escrapea propiedad ATAHUALPA 235 A, La Florida
   Nº5727...inicializando scraping
   Se escrapea propiedad Atahualpa 235 Depto 404-D, La Florida
   Nº5728...inicializando scraping
   Se escrapea propiedad Vicuña Mackena Oriente 6553, La Florida
   Nº5729...inicializando scraping
   Se escrapea propiedad Coronel Godoy
   Nº5730...inicializando scraping
   Se escrapea propiedad Toro Mazote
   Nº5731...inicializando scraping
   Se escrapea propiedad Placilla
   Nº5732...inicializando scraping
   Se escrapea propiedad Buzo Sobenes
   Nº5733...inicializando scraping
   Se escrapea propiedad Avenida Ecuador
   Nº5734...inicializando scraping
   Se escrapea propiedad Constantino
   Nº5735...inicializando scraping
   Se escrapea propiedad San Gumercindo
   Nº5736...inicializando scraping
   Se escrapea propiedad 
   Nº5737...inicializando scraping
   Se escrapea propiedad Recreo
   Nº5738...inicializando scraping
   Se escrapea propiedad CONDE DEL MAULE

   Se escrapea propiedad 
   Nº5827...inicializando scraping
   Se escrapea propiedad Santa Rosa
   Nº5828...inicializando scraping
   Se escrapea propiedad General Gana
Nº5829...la página de la propiedad tardó demasiado en cargar
   Nº5830...inicializando scraping
   Se escrapea propiedad San Francisco
   Nº5831...inicializando scraping
   Se escrapea propiedad General Gana
   Nº5832...inicializando scraping
   Se escrapea propiedad Tarapacá
   Nº5833...inicializando scraping
   Se escrapea propiedad San Diego
   Nº5834...inicializando scraping
   Se escrapea propiedad TARAPACA 1324 DP 504, Santiago
   Nº5835...inicializando scraping
   Se escrapea propiedad Tucapel Jiménez
   Nº5836...inicializando scraping
   Se escrapea propiedad Bombero Adolfo Ossa
   Nº5837...inicializando scraping
   Se escrapea propiedad Lord cochrane 166, Santiago
   Nº5838...inicializando scraping
   Se escrapea propiedad Nueva San Martín
   Nº5839...inicializando scraping
   Se escrapea propiedad ¡Arriendo d

   Nº5925...inicializando scraping
   Se escrapea propiedad Lord Cochrane
   Nº5926...inicializando scraping
   Se escrapea propiedad Gorbea
   Nº5927...inicializando scraping
   Se escrapea propiedad Vergara
   Nº5928...inicializando scraping
   Se escrapea propiedad José Miguel Carrera
   Nº5929...inicializando scraping
   Se escrapea propiedad Almirante Latorre
   Nº5930...inicializando scraping
   Se escrapea propiedad AV.OSSA /AMPLIO Y LUMINOSO DEPTO 2D/2B /METRO PRINCIPE DE GALES
   Nº5931...inicializando scraping
   Se escrapea propiedad Trinidad Ramírez
Nº5932...la página de la propiedad tardó demasiado en cargar
   Nº5933...inicializando scraping
   Se escrapea propiedad DIAGONAL CERVANTES 759 DP 506, Santiago
   Nº5934...inicializando scraping
   Se escrapea propiedad monjitas 360, Santiago
Nº5935...la página de la propiedad tardó demasiado en cargar
La siguiente página terminó de cargar
Se encontraron 29 propiedades en la página nº229
Inicializando scraping de la página nº22

   Se escrapea propiedad Avenida Américo Vespucio
   Nº6023...inicializando scraping
   Se escrapea propiedad Avenida Independencia
   Nº6024...inicializando scraping
   Se escrapea propiedad Jardín Alto
La siguiente página terminó de cargar
Se encontraron 30 propiedades en la página nº232
Inicializando scraping de la página nº232...
   Nº6025...inicializando scraping
   Se escrapea propiedad José Pedro Alessandri 2278 1606, Macul
   Nº6026...inicializando scraping
   Se escrapea propiedad EL GOLF
   Nº6027...inicializando scraping
   Se escrapea propiedad EL GOLF
   Nº6028...inicializando scraping
   Se escrapea propiedad EL GOLF
   Nº6029...inicializando scraping
   Se escrapea propiedad El Golf
   Nº6030...inicializando scraping
   Se escrapea propiedad Callao / Metro Alcántara / Barrio El Golf
   Nº6031...inicializando scraping
   Se escrapea propiedad EL GOLF
   Nº6032...inicializando scraping
   Se escrapea propiedad Metro Alcántara
   Nº6033...inicializando scraping
   Se escrap

In [97]:
seg_904_1136 = seg_904_fin

In [96]:
seg_904_fin

[{'cod_propiedad': 'Código: 101012508',
  'tipo_propiedad': 'Casa',
  'info_general': 'Casa en Venta Usado Colina - Metropolitana Ver ubicación',
  'nombre_propiedad': 'Las Brisas Norte',
  'precio_a': 'UF 14.300',
  'precio_b': '$ 422.271.564',
  'n_dormitorios': '4',
  'n_banos': '4',
  'sup_terreno': '715,9 m²',
  'sup_util': '284,8 m²',
  'sup_construida': '284,8 m²',
  'aval_fiscal': '222.897.150',
  'fecha_publicacion': '07-02-2019',
  'fecha_plusvalia': '2020 1ºSem',
  'valor_plusvalia': '48',
  'latitud': '-33.2164',
  'longitud': '-70.6665'},
 {'cod_propiedad': 'Código: 15510',
  'tipo_propiedad': 'Departamento',
  'info_general': 'Departamento en Venta Usado Las Condes - Metropolitana Ver ubicación',
  'nombre_propiedad': 'Camino Gran Vista',
  'precio_a': 'UF 14.500',
  'precio_b': '$ 428.177.460',
  'n_dormitorios': '4',
  'n_banos': '3',
  'sup_terreno': '',
  'sup_util': '210 m²',
  'sup_construida': '210 m²',
  'aval_fiscal': '216.300.715',
  'fecha_publicacion': '05-04-

**Paso 4**: Se genera un data frame único con cada uno de los segmentos de datos obtenidos.

In [58]:
#Para venta
seg_1_35 #Desde la página 1:35, 1050 propiedades (1050 acumuladas)
seg_36_343 #Desde la página 36:343, 9247 propiedades (10297 acumuladas)
seg_344_416 #Desde la página 344:416, 2190 propiedades (11437 acumuladas)
seg_417_537 #Desde la página 417:537, 3627 propiedades (15064 acumuladas)
seg_538_593 #Desde la página 538:593, 1692 propiedades (16756 acumuladas)
seg_594_903 #Desde la página 594:903, 9279 propiedades (26035 acumuladas)
seg_904_1136 #Desde la página 904:1136, 6989 propiedades (33024 acumuladas)

In [1535]:
#Para arriendos
arr_seg_1_30 #Desde la página 1:30, 935 propiedades (935 acumuladas)
arr_seg_31_234 #Desde la página 31:235, 6114 propiedades (7039 acumuladas)

In [100]:
df = pd.DataFrame(seg_904_1136)
#df

In [103]:
data_casa_1 = pd.read_excel("data_muestra_propiedades_toctoc.xlsx")
data_casa_2 = pd.read_excel("data_muestra_propiedades_toctoc2.xlsx")
data_casa_3 = pd.read_excel("data_muestra_propiedades_toctoc3.xlsx")
data_casa_4 = pd.read_excel("data_muestra_propiedades_toctoc4.xlsx")
data_casa_5 = pd.read_excel("data_muestra_propiedades_toctoc5.xlsx")
data_casa_6 = pd.read_csv("data_muestra_propiedades_toctoc_6.csv")
data_casa_7 = pd.read_csv("data_muestra_propiedades_toctoc_7.csv")

In [107]:
data_arriendo_1 = pd.read_csv("data_muestra_arriendo_propiedades_toctoc_1.csv")
data_arriendo_2 = pd.read_csv("data_muestra_arriendo_propiedades_toctoc_2.csv")

In [104]:
data_total_venta_toctoc=pd.concat([data_casa_1,data_casa_2,data_casa_3,data_casa_4,data_casa_5,data_casa_6,data_casa_7])

In [108]:
data_total_arriendo_toctoc=pd.concat([data_arriendo_1,data_arriendo_2])

In [110]:
df.to_csv('data_total_arriendo_toctoc.csv')

In [1409]:
# Hacemos foco sobre la pestaña principal
driver.switch_to.window(driver.window_handles[0])

In [1039]:
#Crear variable que permita pasar de página con el evento click
pasar_pagina = driver.find_element_by_xpath('//a[@aria-label="Next"]')
pasar_pagina.click()

**Paso 4**: Cerramos el navegador.

In [1724]:
driver.close()